In [5]:
from xml.etree import ElementTree
from xml.dom import minidom
from xml.etree.ElementTree import Element, SubElement, Comment, indent

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, encoding="ISO-8859-1")
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="\t")

In [29]:
import numpy as np
import os
valve_ids = np.arange(2,4+1)
hyb_ids = np.arange(34,36+1)
reg_names = [f'GM{_i}' for _i in np.arange(1,3+1)]

In [26]:
source_folder = r'D:\Shiwei\20210706-P_Forebrain_CTP09_only'
target_drive = r'\\KOLMOGOROV\Chromatin_NAS_5'
imaging_protocol = r'Zscan_750_647_488_s60_n200'
bleach_protocol = r'Bleach_740_647_s5'


In [30]:
cmd_seq = Element('command_sequence')

for _vid, _hid, _rname in zip(valve_ids, hyb_ids, reg_names):
    # comments
    comment = Comment(f"Hyb {_hid} for {_rname}")
    cmd_seq.append(comment)
    # TCEP
    tcep = SubElement(cmd_seq, 'valve_protocol')
    tcep.text = "Flow TCEP"
    # flow adaptor
    adt = SubElement(cmd_seq, 'valve_protocol')
    adt.text = f"Hybridize {_vid}"
    # delay time
    adt_incubation = SubElement(cmd_seq, 'delay')
    adt_incubation.text = "60000"
    
    # change bleach directory
    change_dir = SubElement(cmd_seq, 'change_directory')
    change_dir.text = os.path.join(source_folder, f"Bleach")    
    # wakeup
    wakeup = SubElement(cmd_seq, 'wakeup')
    wakeup.text = "5000"
    # bleach loop
    loop = SubElement(cmd_seq, 'loop', name='Position Loop Zscan', increment="name")
    loop_item = SubElement(loop, 'item', name=bleach_protocol)
    loop_item.text = " "
    
    # wash
    wash = SubElement(cmd_seq, 'valve_protocol')
    wash.text = "Short Wash"
    # readouts
    readouts = SubElement(cmd_seq, 'valve_protocol')
    readouts.text = "Flow Readouts"
    # delay time
    adt_incubation = SubElement(cmd_seq, 'delay')
    adt_incubation.text = "60000"
    
    # change directory
    change_dir = SubElement(cmd_seq, 'change_directory')
    change_dir.text = os.path.join(source_folder, f"H{_hid}{_rname.upper()}")
    # wakeup
    wakeup = SubElement(cmd_seq, 'wakeup')
    wakeup.text = "5000"
    
    # hybridization loop
    loop = SubElement(cmd_seq, 'loop', name='Position Loop Zscan', increment="name")
    loop_item = SubElement(loop, 'item', name=imaging_protocol)
    loop_item.text = " "
    # delay time
    delay = SubElement(cmd_seq, 'delay')
    delay.text = "2000"
    # copy folder
    copy_dir = SubElement(cmd_seq, 'copy_directory')
    source_dir = SubElement(copy_dir, 'source_path')
    source_dir.text = change_dir.text#cmd_seq.findall('change_directory')[-1].text
    target_dir = SubElement(copy_dir, 'target_path')
    target_dir.text = os.path.join(target_drive, 
                                   os.path.basename(os.path.dirname(source_dir.text)), 
                                   os.path.basename(source_dir.text))
    del_source = SubElement(copy_dir, 'delete_source')
    del_source.text = "True"
    # empty line
    indent(target_dir)


In [31]:
print( prettify(cmd_seq))

<?xml version="1.0" ?>
<command_sequence>
	<!--Hyb 34 for GM1-->
	<valve_protocol>Flow TCEP</valve_protocol>
	<valve_protocol>Hybridize 2</valve_protocol>
	<delay>60000</delay>
	<change_directory>D:\Shiwei\20210706-P_Forebrain_CTP09_only\Bleach</change_directory>
	<wakeup>5000</wakeup>
	<loop name="Position Loop Zscan" increment="name">
		<item name="Bleach_740_647_s5"> </item>
	</loop>
	<valve_protocol>Short Wash</valve_protocol>
	<valve_protocol>Flow Readouts</valve_protocol>
	<delay>60000</delay>
	<change_directory>D:\Shiwei\20210706-P_Forebrain_CTP09_only\H34GM1</change_directory>
	<wakeup>5000</wakeup>
	<loop name="Position Loop Zscan" increment="name">
		<item name="Zscan_750_647_488_s60_n200"> </item>
	</loop>
	<delay>2000</delay>
	<copy_directory>
		<source_path>D:\Shiwei\20210706-P_Forebrain_CTP09_only\H34GM1</source_path>
		<target_path>\\KOLMOGOROV\Chromatin_NAS_5\20210706-P_Forebrain_CTP09_only\H34GM1</target_path>
		<delete_source>True</delete_source>
	</copy_directory>
	<